<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/Catboost_20240825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

In [5]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.7 MB/s eta 0:00:00


In [50]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [6]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata2/df_train_allnum4.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata2/df_test_allnum4.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [51]:
df_train.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,CityTier_1,CityTier_2,CityTier_3,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,PreferredPropertyStar_3.0,PreferredPropertyStar_4.0,PreferredPropertyStar_5.0,PitchSatisfactionScore_1,PitchSatisfactionScore_2,PitchSatisfactionScore_3,PitchSatisfactionScore_4,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0,50.00000,1,15.0,0,1.0,4.0,5.0,1,253905.0,1.0,0,0.0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
1,1,56.00000,0,14.0,0,1.0,4.0,2.0,1,404475.0,0.0,1,0.0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
2,2,41.13181,1,10.0,1,1.0,3.0,4.0,0,278145.0,1.0,0,0.0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
3,3,37.00000,1,18.0,1,1.0,3.0,1.0,0,326805.0,0.0,1,0.0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
4,4,48.00000,0,17.0,1,1.0,3.0,4.0,0,258435.0,1.0,1,0.0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0


In [32]:
# Config cell
target = "ProdTaken"

In [33]:

train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [37]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
numerical_features = default_numerical_features

#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=[target, 'id','NumberOfPersonVisiting']
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar']
categorical_features ['TypeofContact', 'Gender', 'Passport', 'ProdTaken', 'HasChildren', 'CityTier_1', 'CityTier_2', 'CityTier_3', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'PreferredPropertyStar_3.0', 'PreferredPropertyStar_4.0', 'PreferredPropertyStar_5.0', 'PitchSatisfactionScore_1', 'PitchSatisfactionScore_2', 'PitchSatisfactionScore_3', 'PitchSatisfactionScore_4', 'PitchSatisfactionScore_5', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior Manager', 'Designation_VP', 'MaritalStatus_未婚', 'MaritalStatus_独身', 'MaritalStatus_結婚', 'MaritalStatus_離婚']
features for training:['Age', 'TypeofContact', 'DurationOfPitch', 'Gender', 'NumberOfFollowups', 'NumberOfT

In [30]:
cols_category=categorical_features

In [42]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [8]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [43]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []

cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=0)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x.iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x.iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

        # キャリブレーション曲線を計算
    fraction_of_positives, mean_predicted_value = calibration_curve(val_y, preds_y_proba, n_bins=10)
    fraction_of_positives_list.append(fraction_of_positives)
    mean_predicted_value_list.append(mean_predicted_value)

0:	test: 0.7614167	best: 0.7614167 (0)	total: 6.12ms	remaining: 6.11s
100:	test: 0.8224259	best: 0.8444529 (8)	total: 622ms	remaining: 5.53s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8444529327
bestIteration = 8

Shrink model to first 9 iterations.
Fold: 1, AUC: 0.8444529326882269, f1 score: 0.7265299375391119 Threshold: 0.3830634471963665
0:	test: 0.8510221	best: 0.8510221 (0)	total: 6.01ms	remaining: 6s
100:	test: 0.8891668	best: 0.8955925 (45)	total: 609ms	remaining: 5.42s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8955925479
bestIteration = 45

Shrink model to first 46 iterations.
Fold: 2, AUC: 0.8955925478695876, f1 score: 0.7765113824815317 Threshold: 0.2525531345811985
0:	test: 0.7704416	best: 0.7704416 (0)	total: 6.05ms	remaining: 6.04s
100:	test: 0.8128774	best: 0.8196481 (53)	total: 633ms	remaining: 5.63s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8196480938
bestIteration = 53

Shrink model to firs

In [44]:
# AUC
print(np.mean(list_metrics_auc), np.mean(list_metrics_f1), np.median(list_cutoff))

0.8421822419094716 0.7233431609847846 0.2655215360915104


In [45]:
import pickle

In [49]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [54]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x79214ecab010>, Proba: [0.02378406 0.03632012 0.04358414 ... 0.07002201 0.044832   0.00992993]
Model: <catboost.core.CatBoostClassifier object at 0x7921a3cc4d60>, Proba: [0.03058637 0.05512255 0.07441935 ... 0.14273045 0.0678563  0.01071107]
Model: <catboost.core.CatBoostClassifier object at 0x79214eb57a00>, Proba: [0.04001313 0.08063496 0.10215214 ... 0.22781194 0.10220003 0.01128892]
Model: <catboost.core.CatBoostClassifier object at 0x792150dd4e80>, Proba: [0.05524576 0.12183673 0.14940658 ... 0.29727614 0.14617814 0.01833543]
Model: <catboost.core.CatBoostClassifier object at 0x79214eca8400>, Proba: [0.06332185 0.13410605 0.19954899 ... 0.37134761 0.16356342 0.01872213]
Model: <catboost.core.CatBoostClassifier object at 0x79214ec6fc10>, Proba: [0.07421418 0.15836582 0.23715629 ... 0.44713873 0.18856127 0.01986167]
Model: <catboost.core.CatBoostClassifier object at 0x79214ec1c190>, Proba: [0.08469022 0.17676823 0.27586279 ... 0.516

In [55]:
count = 0

In [56]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)